In [3]:
using BenchmarkTools
using ForneyLab

In [13]:
# Data
    n_samples = 100
    x = [ t for t = 1:n_samples ] # True state
    y = x + sqrt(200.0) * randn(n_samples); # Noisy observations of state 
    
    g = FactorGraph()

    # declare priors as random variables
    @RV m_x_t_min # m_x_t_min = Variable(id=:m_x_t_min)
    @RV v_x_t_min # v_x_t_min = Variable(id=:v_x_t_min)
    @RV x_t_min ~ GaussianMeanVariance(m_x_t_min, v_x_t_min)

    # System equations
    # u = 1.0; v = 200.0
    @RV n_t ~ GaussianMeanVariance(0.0, 200.0)
    @RV x_t = x_t_min + 1.0
    @RV y_t = x_t + n_t

    # Name variable for ease of lookup
    #x_t.id = :x_t;
    
    # Placeholders for prior
    placeholder(m_x_t_min, :m_x_t_min) # placeholder(:m_x_t_min) does not work
    placeholder(v_x_t_min, :v_x_t_min)

    # Placeholder for data
    placeholder(y_t, :y_t);
    
    algo = sumProductAlgorithm(x_t) # Figure out a schedule and compile to Julia code
    
    # Define algorithm
    eval(Meta.parse(algorithmSourceCode(algo)))

function forneylab()
    
    # Define values for prior statistics
    m_x_0 = 0.0
    v_x_0 = 1000.0

    m_x = Vector{Float64}(undef, n_samples)
    v_x = Vector{Float64}(undef, n_samples)

    m_x_t_min = m_x_0
    v_x_t_min = v_x_0
    for t = 1:n_samples
        # Prepare data and prior statistics
        data = Dict(:y_t       => y[t],
                    :m_x_t_min => m_x_t_min,
                    :v_x_t_min => v_x_t_min)

        # Execute algorithm
        marginals = step!(data)

        # Extract posterior statistics
        m_x_t_min = mean(marginals[:x_t])
        v_x_t_min = var(marginals[:x_t])

        # Store to buffer 
        m_x[t] = m_x_t_min
        v_x[t] = v_x_t_min
    end
    
    m_x, v_x
end

forneylab (generic function with 1 method)

([17.71620279903645, 9.246770880736602, 0.8841633487294476, 7.629711025853187, 12.92868164935897, 13.962855784054621, 14.286715381646005, 12.392132942056424, 14.61448593160173, 13.809742528258552  …  91.34313091201793, 92.39441650496143, 93.34731376616058, 94.34376563305833, 95.47239776793931, 96.2026669815506, 97.21213604211245, 98.04144681559835, 98.88754010695347, 99.8842641256655], [166.66666666666666, 90.90909090909092, 62.5, 47.61904761904761, 38.46153846153846, 32.25806451612903, 27.77777777777778, 24.39024390243903, 21.739130434782613, 19.607843137254907  …  2.1929824561403493, 2.169197396963122, 2.1459227467811144, 2.123142250530784, 2.1008403361344525, 2.079002079002078, 2.057613168724279, 2.0366598778004064, 2.0161290322580636, 1.9960079840319351])

In [9]:
using Cassette

In [10]:
Cassette.@context AdditionDebugContext

Cassette.Context{nametype(AdditionDebugContext),M,T,P,B,H} where H<:Union{Cassette.DisableHooks, Nothing} where B<:Union{Nothing, IdDict{Module,Dict{Symbol,Cassette.BindingMeta}}} where P<:Cassette.AbstractPass where T<:Union{Nothing, Cassette.Tag} where M

In [18]:
Cassette.prehook(::AdditionDebugContext, ::typeof(ForneyLab.SPAdditionOutNGP), args...) = begin
     println("ruleSPAdditionOutNGG: ", args...)
end

In [19]:
Cassette.overdub(AdditionDebugContext(), forneylab)

ruleSPAdditionOutNGG: UndefInitializer()100
ruleSPAdditionOutNGG: UndefInitializer()100
ruleSPAdditionOutNGG: 1100
ruleSPAdditionOutNGG: 1
ruleSPAdditionOutNGG: y_t21.05944335884374
ruleSPAdditionOutNGG: m_x_t_min0.0
ruleSPAdditionOutNGG: v_x_t_min1000.0
ruleSPAdditionOutNGG: (:y_t => 21.05944335884374, :m_x_t_min => 0.0, :v_x_t_min => 1000.0)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOu

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 90.90909090909092)
ruleSPAdditionOutNGG: δ(m=90.91)

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (9.246770880736602, 90.90909090909092)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m9.246770880736602
ruleSPAdditionOutNGG: v90.90909090909092
ruleSPAdditionOutNGG: (:m => 9.246770880736602, :v => 90.90909090909092)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 9.246770880736602,:v => 90.90909090909092)
ruleSPAdditionOutNGG: 𝒩

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (0.0, 200.0)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m0.0
ruleSPAdditionOutNGG: v200.0
ruleSPAdditionOutNGG: (:m => 0.0, :v => 200.0)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 0.0,:v => 200.0)
ruleSPAdditionOutNGG: 𝒩(m=0.00, v=2.00e+02)

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (26.01546359264916,)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m26.01546359264916
ruleSPAdditionOutNGG: (:m => 26.01546359264916,)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdd

ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:w => 1.0,:xi => 0.0)
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 1
ruleSPAdditionOutNGG: y_t14.140561284472007
ruleSPAdditionOutNGG: m_x_t_min12.92868164935897
ruleSPAdditionOutNGG: v_x_t_min38.46153846153846
ruleSPAdditionOutNGG: (:y_t => 14.140561284472007, :m_x_t_min => 12.9286816493589

ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: UndefInitializer()4
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (13.962855784054621,)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m13.962855784054621
ruleSPAdditionOutNGG: (:m => 13.962855784054621,)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 13.962855784054621)
ruleSPAdditionOutNGG: δ(m=13.96)

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (32.25806451612903,)
ruleSPAdditionOutNGG: ()
rul

...

ruleSPAdditionOutNGG: (:m => 0.0,)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 0.0)
ruleSPAdditionOutNGG: δ(m=0.00)

ruleSPAdditionOutNGG: (200.0,)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m200.0
ruleSPAdditionOutNGG: (:m => 200.0,)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 200.0)
ruleSPAdditionOutNGG: δ(m=2.00e+02)

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdd

ruleSPAdditionOutNGG: Dict(:m => 82.71549863073011)
ruleSPAdditionOutNGG: δ(m=82.72)

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (82.71549863073011, 200.0)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m82.71549863073011
ruleSPAdditionOutNGG: v200.0
ruleSPAdditionOutNGG: (:m => 82.71549863073011, :v => 200.0)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 82.71549863073011,:v => 200.0)
ruleSPAdditionOutNGG: 𝒩(m=82.72, v=2.00e+02)

ruleSPAdditionOutNGG: (0.0, 1.0)
ruleSPAdditionOutNGG: xi0.0
ruleSPAdditionOutNGG: w1.0
ruleSPAdditionOutNGG: (:xi => 0.0, :w => 1.0)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: Undef

Excessive output truncated after 524298 bytes.


ruleSPAdditionOutNGG: (:m => 66.55747354859902,)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: Dict(:m => 66.55747354859902)
ruleSPAdditionOutNGG: δ(m=66.56)

ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: (66.55747354859902, 200.0)
ruleSPAdditionOutNGG: ()
ruleSPAdditionOutNGG: m66.55747354859902
ruleSPAdditionOutNGG: v200.0
ruleSPAdditionOutNGG: (:m => 66.55747354859902, :v => 200.0)
ruleSPAdditionOutNGG: 
ruleSPAdditionOutNGG: UndefInitializer()(16,)
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 0
ruleSPAdditionOutNGG: 16
ruleSPAdditionOutNGG: UndefInitializer()16
ruleSPAdditionOutNGG: UndefInitializer()

([17.71620279903645, 9.246770880736602, 0.8841633487294476, 7.629711025853187, 12.92868164935897, 13.962855784054621, 14.286715381646005, 12.392132942056424, 14.61448593160173, 13.809742528258552  …  91.34313091201793, 92.39441650496143, 93.34731376616058, 94.34376563305833, 95.47239776793931, 96.2026669815506, 97.21213604211245, 98.04144681559835, 98.88754010695347, 99.8842641256655], [166.66666666666666, 90.90909090909092, 62.5, 47.61904761904761, 38.46153846153846, 32.25806451612903, 27.77777777777778, 24.39024390243903, 21.739130434782613, 19.607843137254907  …  2.1929824561403493, 2.169197396963122, 2.1459227467811144, 2.123142250530784, 2.1008403361344525, 2.079002079002078, 2.057613168724279, 2.0366598778004064, 2.0161290322580636, 1.9960079840319351])